## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,80.19,85,93,4.85,light rain
1,1,Hermanus,ZA,-34.4187,19.2345,66.15,84,52,15.28,light rain
2,2,Sorland,NO,67.6670,12.6934,34.30,61,100,13.29,light rain
3,3,Busselton,AU,-33.6500,115.3333,64.85,73,73,12.88,broken clouds
4,4,Yulara,AU,-25.2406,130.9889,78.64,19,0,6.91,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,80.19,85,93,4.85,light rain
4,4,Yulara,AU,-25.2406,130.9889,78.64,19,0,6.91,clear sky
7,7,Cartagena,CO,10.3997,-75.5144,76.62,94,0,4.61,clear sky
13,13,Nabire,ID,-3.3667,135.4833,78.91,87,92,2.93,moderate rain
16,16,Srandakan,ID,-7.9386,110.2506,77.31,86,100,3.69,overcast clouds
17,17,Alofi,NU,-19.0595,-169.9187,76.89,78,91,10.36,light rain
27,27,Georgetown,MY,5.4112,100.3354,78.75,83,40,6.91,scattered clouds
28,28,Quatre Cocos,MU,-20.2078,57.7625,84.31,68,21,9.42,few clouds
31,31,Tarakan,ID,3.3000,117.6333,80.10,89,40,5.75,scattered clouds
32,32,Tautira,PF,-17.7333,-149.1500,78.82,81,84,17.76,light rain


In [5]:
# 4a. Determine if there are any empty rows.
import numpy as np
np.where(pd.isnull(preferred_cities_df))

(array([ 97, 157], dtype=int64), array([2, 2], dtype=int64))

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,80.19,85,93,4.85,light rain
4,4,Yulara,AU,-25.2406,130.9889,78.64,19,0,6.91,clear sky
7,7,Cartagena,CO,10.3997,-75.5144,76.62,94,0,4.61,clear sky
13,13,Nabire,ID,-3.3667,135.4833,78.91,87,92,2.93,moderate rain
16,16,Srandakan,ID,-7.9386,110.2506,77.31,86,100,3.69,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
558,558,Muli,MV,2.9167,73.5667,82.44,70,92,15.03,overcast clouds
560,560,Vila Do Maio,CV,15.1333,-23.2167,76.10,74,91,22.57,overcast clouds
563,563,Praya,CV,14.9215,-23.5087,79.34,61,20,28.77,few clouds
564,564,Anloga,GH,5.7947,0.8973,85.03,70,70,9.15,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df.dropna()[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,
4,Yulara,AU,78.64,clear sky,-25.2406,130.9889,
7,Cartagena,CO,76.62,clear sky,10.3997,-75.5144,
13,Nabire,ID,78.91,moderate rain,-3.3667,135.4833,
16,Srandakan,ID,77.31,overcast clouds,-7.9386,110.2506,
17,Alofi,NU,76.89,light rain,-19.0595,-169.9187,
27,Georgetown,MY,78.75,scattered clouds,5.4112,100.3354,
28,Quatre Cocos,MU,84.31,few clouds,-20.2078,57.7625,
31,Tarakan,ID,80.10,scattered clouds,3.3000,117.6333,
32,Tautira,PF,78.82,light rain,-17.7333,-149.1500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
4,Yulara,AU,78.64,clear sky,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
7,Cartagena,CO,76.62,clear sky,10.3997,-75.5144,Hotel Colombia Real - Cartagena
13,Nabire,ID,78.91,moderate rain,-3.3667,135.4833,Hotel Nusantara Nabire
16,Srandakan,ID,77.31,overcast clouds,-7.9386,110.2506,Puri Brata
...,...,...,...,...,...,...,...
558,Muli,MV,82.44,overcast clouds,2.9167,73.5667,Medhufushi Island Resort
560,Vila Do Maio,CV,76.10,overcast clouds,15.1333,-23.2167,Hotel Marilu
563,Praya,CV,79.34,few clouds,14.9215,-23.5087,LT Aparthotel
564,Anloga,GH,85.03,broken clouds,5.7947,0.8973,Pin Drop Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.dropna().to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.dropna().iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df.dropna()[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))